Importing required Libraries

In [1]:
import numpy as np
import pandas as pd
import re

######### Raw Data Dump

In [2]:
twitter_data = pd.read_csv("Tweets_Sep_Pfizer.csv")

In [3]:
twitter_data.head(2)

,Unnamed: 0,Id,Timestamp,Source,Retwwet_Count,User_Name,Location,Tweets
0,0,1441380234113081350,2021-09-24 12:33:22,Twitter Web App,0,X_LOLOLOLOLOL_X,NaN,FDA permits use of the Pfizer-BioNTech Covid v...
1,1,1441380226131390472,2021-09-24 12:33:20,Twitter Web App,0,Jack Schmidt,NaN,Pfizer CEO Bourla asks employees to join the s...


In [4]:
twitter_data = twitter_data.rename(columns = {'Tweets':'tweets'})

In [5]:
twitter_data=pd.DataFrame(twitter_data)

In [6]:
twitter_data.shape

(1217, 8)

In [7]:
twitter_data=twitter_data.drop(columns = 'Location', axis = 1)

In [8]:
twitter_data= twitter_data.drop_duplicates()

In [9]:
twitter_data.shape

(1217, 7)

In [10]:
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].apply(str)

Pre Processing - Removing unwanted Characters

In [11]:
twitter_data.head(2)

,Unnamed: 0,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,0,1441380234113081350,2021-09-24 12:33:22,Twitter Web App,0,X_LOLOLOLOLOL_X,FDA permits use of the Pfizer-BioNTech Covid v...
1,1,1441380226131390472,2021-09-24 12:33:20,Twitter Web App,0,Jack Schmidt,Pfizer CEO Bourla asks employees to join the s...


Pre Processing
Remove Punctuations

In [12]:
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'https(\S)+', r' ')
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'http ...', r' ')

twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r' ')
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace(r'@[\S]+',r' ')

twitter_data.iloc[:,5] = twitter_data.iloc[:,5].str.replace('http', '')

In [13]:
twitter_data.head(5)

,Unnamed: 0,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,0,1441380234113081350,2021-09-24 12:33:22,Twitter Web App,0,X_LOLOLOLOLOL_X,FDA permits use of the Pfizer-BioNTech Covid v...
1,1,1441380226131390472,2021-09-24 12:33:20,Twitter Web App,0,Jack Schmidt,Pfizer CEO Bourla asks employees to join the s...
2,2,1441380207655460865,2021-09-24 12:33:15,Twitter for iPhone,0,Fred Zarnowski,@WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3,3,1441380207517003781,2021-09-24 12:33:15,Twitter Web App,0,The_Real_Fly,Pfizer in talks over full license for COVID-19...
4,4,1441380151955128325,2021-09-24 12:33:02,Twitter Web App,0,UNC Otolaryngology/Head & Neck Surgery,"🎉Congrats to @ProfBrentSenior, who will receiv..."


In [14]:
PUNCT_TO_REMOVE = '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`'

In [15]:
def remove_punctuation(text):
    return text.translate(str.maketrans(' ', ' ', PUNCT_TO_REMOVE))

In [16]:
twitter_data.iloc[:,5] = twitter_data.iloc[:,5].apply(lambda text: remove_punctuation(text))

In [17]:
all_words = list(twitter_data.iloc[:,5].str.lower().str.split(' ', expand=True).stack().unique())

In [18]:
twitter_data['tweets'].head(10)

0    FDA permits use of the Pfizer-BioNTech Covid v...
1    Pfizer CEO Bourla asks employees to join the s...
2    @WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3    Pfizer in talks over full license for COVID-19...
4    🎉Congrats to @ProfBrentSenior, who will receiv...
5    I guess @pfizer 💰 does talk after all huh @CDC...
6    @rec777777 @BanterNative @iancrichton4 @MalCha...
7    @ell_enn Ma'am, I heard that one can get Pfize...
8    @debjshaw Anyone with compromised immunity at ...
9    @Dovey08261451 @V2019N There aren't specifics....
Name: tweets, dtype: object

Remove most occuring words

In [19]:
from collections import Counter
cnt = Counter()
for text in twitter_data['tweets'].values:
    for word in text.split():
        cnt[word] += 1

In [20]:
cnt.most_common(16)

[('the', 879),
 ('Pfizer', 728),
 ('and', 642),
 ('for', 600),
 ('to', 585),
 ('of', 495),
 ('a', 426),
 ('in', 336),
 ('is', 278),
 ('I', 274),
 ('at', 253),
 ('booster', 244),
 ('CDC', 211),
 ('vaccine', 204),
 ('on', 196),
 ('that', 192)]

In [21]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(25)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

In [22]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_freqwords(text))

In [23]:
twitter_data['tweets'].head(10)

0    FDA permits use Pfizer-BioNTech Covid kids age...
1    CEO Bourla asks employees join scrum D.C. drug...
2    @WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3      talks over full license COVID-19 Singapore $PFE
4    🎉Congrats @ProfBrentSenior, who will receive a...
5    guess @pfizer 💰 does talk after all huh @CDCDi...
6    @rec777777 @BanterNative @iancrichton4 @MalCha...
7    @ell_enn Ma'am, heard one can irrespective age...
8    @debjshaw Anyone compromised immunity any age ...
9    @Dovey08261451 @V2019N There aren't specifics....
Name: tweets, dtype: object

Remove rare words

In [24]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [25]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_rarewords(text))

In [26]:
twitter_data['tweets'].head(10)

0    FDA permits use Pfizer-BioNTech Covid kids age...
1    CEO Bourla asks employees join scrum D.C. drug...
2    @WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3      talks over full license COVID-19 Singapore $PFE
4    🎉Congrats @ProfBrentSenior, who will receive a...
5    guess @pfizer 💰 does talk after all huh @CDCDi...
6    @rec777777 @BanterNative @iancrichton4 @MalCha...
7    @ell_enn Ma'am, heard one can irrespective age...
8    @debjshaw Anyone compromised immunity any age ...
9    @Dovey08261451 @V2019N There aren't specifics....
Name: tweets, dtype: object

Lemmatize

In [27]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [28]:
import nltk
#nltk.download('book')

#nltk.download('all-corpora')

In [29]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [30]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: lemmatize_words(text))

In [31]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

In [32]:
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

In [33]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: lemmatize_words(text))

In [34]:
twitter_data.head(10)

,Unnamed: 0,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,0,1441380234113081350,2021-09-24 12:33:22,Twitter Web App,0,XLOLOLOLOLOLX,FDA permit use Pfizer-BioNTech Covid kid age 1...
1,1,1441380226131390472,2021-09-24 12:33:20,Twitter Web App,0,Jack Schmidt,CEO Bourla ask employee join scrum D.C. drug-p...
2,2,1441380207655460865,2021-09-24 12:33:15,Twitter for iPhone,0,Fred Zarnowski,@WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3,3,1441380207517003781,2021-09-24 12:33:15,Twitter Web App,0,TheRealFly,talk over full license COVID-19 Singapore $PFE
4,4,1441380151955128325,2021-09-24 12:33:02,Twitter Web App,0,UNC OtolaryngologyHead Neck Surgery,"🎉Congrats @ProfBrentSenior, who will receive a..."
5,5,1441380143105142799,2021-09-24 12:33:00,Twitter for iPhone,0,Jason,guess @pfizer 💰 doe talk after all huh @CDCDir...
6,6,1441380136738189327,2021-09-24 12:32:58,Twitter Web App,0,stevenscraig,@rec777777 @BanterNative @iancrichton4 @MalCha...
7,7,1441380071294455809,2021-09-24 12:32:43,Twitter for Android,0,faridahmedkhan,"@ell_enn Ma'am, hear one can irrespective age/..."
8,8,1441380034741112841,2021-09-24 12:32:34,Twitter for iPhone,0,Stephani Shelton,@debjshaw Anyone compromise immunity any age h...
9,9,1441380001320951809,2021-09-24 12:32:26,Twitter Web App,0,Infectious Diseases,@Dovey08261451 @V2019N There aren't specifics....


Removal of Stopwords

In [35]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [36]:
STOPWORDS = set(stopwords.words('english'))

In [37]:
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [38]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_stopwords(text))

In [39]:
twitter_data['tweets'].head(10)

0    FDA permit use Pfizer-BioNTech Covid kid age 1...
1    CEO Bourla ask employee join scrum D.C. drug-p...
2    @WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3            talk full license COVID-19 Singapore $PFE
4    🎉Congrats @ProfBrentSenior, receive ACS/Pfizer...
5    guess @pfizer 💰 doe talk huh @CDCDirector You’...
6    @rec777777 @BanterNative @iancrichton4 @MalCha...
7    @ell_enn Ma'am, hear one irrespective age/nece...
8    @debjshaw Anyone compromise immunity age ha ab...
9    @Dovey08261451 @V2019N There specifics. It's b...
Name: tweets, dtype: object

Removal of unwanted Emojis

In [40]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF" 
                           u"\U0001F1E0-\U0001F1FF" 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [41]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: remove_emoji(text))

In [42]:
twitter_data['tweets'].head(5)

0    FDA permit use Pfizer-BioNTech Covid kid age 1...
1    CEO Bourla ask employee join scrum D.C. drug-p...
2    @WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3            talk full license COVID-19 Singapore $PFE
4    Congrats @ProfBrentSenior, receive ACS/Pfizer ...
Name: tweets, dtype: object

********** Converting Emoticons to Words

In [43]:
import emoji

*** Defining Emoticons

In [44]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [45]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

In [46]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: convert_emoticons(text))

In [47]:
twitter_data['tweets'].head(5)

0    FDA permit use Pfizer-BioNTech Covid kid age 1...
1    CEO Bourla ask employee join scrum D.C. drug-p...
2    @WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3            talk full license COVID-19 Singapore $PFE
4    Congrats @ProfBrentSenior, receive ACS/Pfizer ...
Name: tweets, dtype: object

Converting Chat Words

**** Defining the Dictionary

In [48]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [49]:
chat_words_map_dict = {}
chat_words_list = []

In [50]:
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

In [51]:
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [52]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: chat_words_conversion(text))

In [53]:
twitter_data['tweets'].head(10)

0    FDA permit use Pfizer-BioNTech Covid kid age 1...
1    CEO Bourla ask employee join scrum D.C. drug-p...
2    @WesElyMD @B52Malmet @CDCgov @CDCDirector I’m ...
3            talk full license COVID-19 Singapore $PFE
4    Congrats @ProfBrentSenior, receive ACS/Pfizer ...
5    guess @pfizer doe talk huh @CDCDirector You’re...
6    @rec777777 @BanterNative @iancrichton4 @MalCha...
7    @ell_enn Ma'am, hear one irrespective age/nece...
8    @debjshaw Anyone compromise immunity age ha ab...
9    @Dovey08261451 @V2019N There specifics. It's b...
Name: tweets, dtype: object

Some more Cleansing using RE Library

In [54]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = text.lower()       
    return text

In [55]:
twitter_data['tweets'] = twitter_data['tweets'].apply(lambda text: clean_text(text))

In [56]:
twitter_data['tweets'].head(10)

0    fda permit use pfizer biontech covid kid age  ...
1    ceo bourla ask employee join scrum d c  drug p...
2     weselymd  b  malmet  cdcgov  cdcdirector i m ...
3            talk full license covid    singapore  pfe
4    congrats  profbrentsenior  receive acs pfizer ...
5    guess  pfizer doe talk huh  cdcdirector you re...
6     rec        banternative  iancrichton   malcha...
7     ell enn ma'am  hear one irrespective age nece...
8     debjshaw anyone compromise immunity age ha ab...
9     dovey          v    n there specifics  it's b...
Name: tweets, dtype: object

In [57]:
num_words = twitter_data['tweets'].apply(lambda x: len(x.split()))
num_words_mean, num_words_std = np.mean(num_words), np.std(num_words)

num_sentences = twitter_data['tweets'].apply(lambda x: len(re.split( '~ ...' ,'~'.join(x.split('.')))))
num_sentences_mean = np.mean(num_sentences)

Remove extra spaces and Words less than 3 letters

In [58]:
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'[ ]{2, }',r' ')
twitter_data['tweets'] = twitter_data['tweets'].str.replace(r'\b\w{1,3}\b', r'')

In [59]:
twitter_data.head(10)

,Unnamed: 0,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets
0,0,1441380234113081350,2021-09-24 12:33:22,Twitter Web App,0,XLOLOLOLOLOLX,permit pfizer biontech covid httpssk...
1,1,1441380226131390472,2021-09-24 12:33:20,Twitter Web App,0,Jack Schmidt,bourla employee join scrum drug pricing f...
2,2,1441380207655460865,2021-09-24 12:33:15,Twitter for iPhone,0,Fred Zarnowski,weselymd malmet cdcgov cdcdirector per...
3,3,1441380207517003781,2021-09-24 12:33:15,Twitter Web App,0,TheRealFly,talk full license covid singapore
4,4,1441380151955128325,2021-09-24 12:33:02,Twitter Web App,0,UNC OtolaryngologyHead Neck Surgery,congrats profbrentsenior receive pfizer int...
5,5,1441380143105142799,2021-09-24 12:33:00,Twitter for iPhone,0,Jason,guess pfizer talk cdcdirector vaccine ...
6,6,1441380136738189327,2021-09-24 12:32:58,Twitter Web App,0,stevenscraig,banternative iancrichton malchandl...
7,7,1441380071294455809,2021-09-24 12:32:43,Twitter for Android,0,faridahmedkhan,' hear irrespective necessity fees ne...
8,8,1441380034741112841,2021-09-24 12:32:34,Twitter for iPhone,0,Stephani Shelton,debjshaw anyone compromise immunity able ...
9,9,1441380001320951809,2021-09-24 12:32:26,Twitter Web App,0,Infectious Diseases,dovey there specifics ' broad ...


In [60]:
twitter_data['text_length'] = [len(text.split(' ')) for text in twitter_data.tweets]
print(twitter_data.shape)

(1217, 8)


Dropping texts with length <3 and drop duplicates

In [61]:
twitter_data = twitter_data[twitter_data['text_length']>2]
twitter_data = twitter_data.drop_duplicates(subset=['tweets'])

print(twitter_data.shape)

(1212, 8)


In [62]:
twitter_data.head(10)

,Unnamed: 0,Id,Timestamp,Source,Retwwet_Count,User_Name,tweets,text_length
0,0,1441380234113081350,2021-09-24 12:33:22,Twitter Web App,0,XLOLOLOLOLOLX,permit pfizer biontech covid httpssk...,24
1,1,1441380226131390472,2021-09-24 12:33:20,Twitter Web App,0,Jack Schmidt,bourla employee join scrum drug pricing f...,22
2,2,1441380207655460865,2021-09-24 12:33:15,Twitter for iPhone,0,Fred Zarnowski,weselymd malmet cdcgov cdcdirector per...,26
3,3,1441380207517003781,2021-09-24 12:33:15,Twitter Web App,0,TheRealFly,talk full license covid singapore,10
4,4,1441380151955128325,2021-09-24 12:33:02,Twitter Web App,0,UNC OtolaryngologyHead Neck Surgery,congrats profbrentsenior receive pfizer int...,50
5,5,1441380143105142799,2021-09-24 12:33:00,Twitter for iPhone,0,Jason,guess pfizer talk cdcdirector vaccine ...,33
6,6,1441380136738189327,2021-09-24 12:32:58,Twitter Web App,0,stevenscraig,banternative iancrichton malchandl...,232
7,7,1441380071294455809,2021-09-24 12:32:43,Twitter for Android,0,faridahmedkhan,' hear irrespective necessity fees ne...,17
8,8,1441380034741112841,2021-09-24 12:32:34,Twitter for iPhone,0,Stephani Shelton,debjshaw anyone compromise immunity able ...,33
9,9,1441380001320951809,2021-09-24 12:32:26,Twitter Web App,0,Infectious Diseases,dovey there specifics ' broad ...,45


In [63]:
twitter_data.to_csv('Processed Data - Pfizer.csv')